* LightGBM 似乎是最有效的模型
* 添加特征进去模型效果不下降, 反而提升
* 所有把两时间窗口之差的模型, 拿出来在跑跑看, 只测试 LightGBM
* 单纯增加时间窗口之差, 模型效果下降了

In [45]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import scipy as sp

import warnings
warnings.filterwarnings('ignore')

In [35]:
train = pd.read_csv('../../raw/LiChuan/dataset.csv')

In [36]:
cal_column = ['sale_quantity_this_last_month', 'sale_quantity_last_2M_sum', 'sale_quantity_last_3M_sum', 'sale_quantity_last_6M_sum', 'sale_quantity_last_12M_sum']

In [37]:
total_colunms = 9940
def sale_data_diff(train,cal_column):
    train_diff = pd.DataFrame()
    train_diff = train.loc[:,['class_id','sale_date',cal_column]]
    train_diff = train_diff[140:9940].reset_index().drop(['index'],axis = 1)
    str = cal_column +'_diff'
    train_diff[str] =  pd.DataFrame({str:train[140:total_colunms]['sale_quantity_last_3M_sum'].values - train[0:(total_colunms-140)]['sale_quantity_last_3M_sum'].values})
    train_diff = train_diff.drop([cal_column],axis = 1)
    return train_diff

In [38]:
for i in cal_column:
    train_diff = sale_data_diff(train,i)
    train = pd.merge(train, train_diff, on=['class_id','sale_date'], how='left')
    
train.fillna(0, inplace=True)

In [39]:
# 去掉 2012 年数据
train = train[train['year'] != 2012]
labels = train['sale_quantity']
train = train.drop(['sale_date', 'class_id', 'sale_quantity'], axis=1)

In [40]:
# 训练集 2013-01 至 2017-07, 测试集 2017-08
train_1 = train[:-560]
test_1 = train[-560:-420]
labels_1 = labels[:-560]
test_labels_1 = labels[-560:-420]

# 训练集 2013-01 至 2017-08, 测试集 2017-09
train_2 = train[:-420]
test_2 = train[-420:-280]
labels_2 = labels[:-420]
test_labels_2 = labels[-420:-280]

# 训练集 2013-01 至 2017-09, 测试集 2017-10
train_3 = train[:-280]
test_3 = train[-280:-140]
labels_3 = labels[:-280]
test_labels_3 = labels[-280:-140]

In [46]:
lgb_train_1 = lgb.Dataset(train_1, labels_1)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_1 = lgb.train(params, lgb_train_1, 900)
lgb_pred_1 = model_1.predict(test_1)
rmsetmp_1 = sp.sqrt(sp.mean((test_labels_1 - lgb_pred_1) ** 2))

print('This time rmse is: '+ str(rmsetmp_1))

This time rmse is: 134.862765044


In [47]:
lgb_train_2 = lgb.Dataset(train_2, labels_2)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_2 = lgb.train(params, lgb_train_2, 900)
lgb_pred_2 = model_2.predict(test_2)
rmsetmp_2 = sp.sqrt(sp.mean((test_labels_2 - lgb_pred_2) ** 2))

print('This time rmse is: '+ str(rmsetmp_2))

This time rmse is: 118.204111261


In [48]:
lgb_train_3 = lgb.Dataset(train_3, labels_3)

params = {
'learning_rate': 0.003,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 30,
'min_hessian': 1,
'verbose': -1,}

model_3 = lgb.train(params, lgb_train_3, 900)
lgb_pred_3 = model_3.predict(test_3)
rmsetmp_3 = sp.sqrt(sp.mean((test_labels_3 - lgb_pred_3) ** 2))

print('This time rmse is: '+ str(rmsetmp_3))

This time rmse is: 128.105316159


In [49]:
print('The average rmse is: '+ str((rmsetmp_1 + rmsetmp_2 + rmsetmp_3)/3))

The average rmse is: 127.057397488


* 比不加时间窗口之差的模型效果更差